In [2]:
!gdalbuildvrt output.vrt ./chunked_rasters/*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
import os
from osgeo import ogr, gdal, gdalconst
from multiprocessing import Pool, cpu_count

def rasterize_chunk(vector_file, raster_template, chunk_extents, output_raster, burn_value=1):
    """Rasterizes a chunk of the vector file based on specified extents."""
    raster_template_ds = gdal.Open(raster_template, gdalconst.GA_ReadOnly)
    if raster_template_ds is None:
        raise FileNotFoundError(f"Unable to open raster template: {raster_template}")
    
    geotransform = raster_template_ds.GetGeoTransform()
    projection = raster_template_ds.GetProjection()
    pixel_width = geotransform[1]
    pixel_height = -geotransform[5]
    
    # Calculate output raster dimensions
    x_min, x_max, y_min, y_max = chunk_extents
    xsize = int((x_max - x_min) / pixel_width)
    ysize = int((y_max - y_min) / pixel_height)
    
    # Create the output raster for this chunk
    driver = gdal.GetDriverByName('GTiff')
    dst_ds = driver.Create(output_raster, xsize, ysize, 1, gdalconst.GDT_Byte, options=["COMPRESS=DEFLATE", "PREDICTOR=2", "ZLEVEL=9"])
    dst_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, -pixel_height))
    dst_ds.SetProjection(projection)
    
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.SetNoDataValue(0)
    dst_band.Fill(0)
    
    # Rasterize the vector layer into this chunk
    vector_ds = ogr.Open(vector_file)
    vector_layer = vector_ds.GetLayer()
    gdal.RasterizeLayer(dst_ds, [1], vector_layer, burn_values=[burn_value])
    
    # Close datasets
    dst_ds = None
    raster_template_ds = None
    vector_ds = None
    print(f"Processed chunk: {output_raster}")


def split_into_chunks(extents, chunk_size):
    """Splits the full raster extents into smaller chunks for parallel processing."""
    x_min, x_max, y_min, y_max = extents
    chunks = []
    x = x_min
    while x < x_max:
        y = y_min
        while y < y_max:
            chunk_x_max = min(x + chunk_size, x_max)
            chunk_y_max = min(y + chunk_size, y_max)
            chunks.append((x, chunk_x_max, y, chunk_y_max))
            y += chunk_size
        x += chunk_size
    return chunks

def parallel_rasterize_to_chunks(vector_file, raster_template, output_folder, chunk_size, burn_value=1):
    """Rasterizes the vector file to the given raster resolution using parallel processing."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    raster_template_ds = gdal.Open(raster_template, gdalconst.GA_ReadOnly)
    if raster_template_ds is None:
        raise FileNotFoundError(f"Unable to open raster template: {raster_template}")
    
    geotransform = raster_template_ds.GetGeoTransform()
    xsize, ysize = raster_template_ds.RasterXSize, raster_template_ds.RasterYSize
    
    x_min = geotransform[0]
    x_max = x_min + xsize * geotransform[1]
    y_max = geotransform[3]
    y_min = y_max + ysize * geotransform[5]
    
    raster_template_ds = None  # Close the template raster
    
    # Split the raster extents into chunks
    chunks = split_into_chunks((x_min, x_max, y_min, y_max), chunk_size)
    temp_rasters = [os.path.join(output_folder, f"chunk_{i}.tif") for i in range(len(chunks))]
    
    # Prepare arguments for parallel processing
    args = [
        (vector_file, raster_template, chunks[i], temp_rasters[i], burn_value)
        for i in range(len(chunks))
    ]
    
    # Process chunks in parallel
    with Pool(cpu_count()) as pool:
        pool.starmap(rasterize_chunk, args)
    
    print(f"Rasterization complete. Chunked rasters saved in {output_folder}")

# Example usage
vector_file = "/media/dave/3230-6239/IACS_20240715/iacs_lines.shp"
raster_template = "/media/dave/windows_OS/lidar/Export/6-Above15m_compressed.tif"
output_folder = "./chunked_rasters"
chunk_size = 10000  # 10km by 10km chunks

parallel_rasterize_to_chunks(vector_file, raster_template, output_folder, chunk_size, burn_value=1)


In [ ]:
#virtual merge rasters

In [ ]:
!gdalbuildvrt output.vrt ./chunked_rasters/*.tif

In [3]:
import os
import numpy as np
from osgeo import gdal, gdalconst
from multiprocessing import Pool, cpu_count


def process_chunk_parallel(args):
    """Worker function to process a single chunk in parallel."""
    src_raster, x, y, x_end, y_end, min_threshold, max_threshold, nodata = args
    src_ds = gdal.Open(src_raster, gdalconst.GA_ReadOnly)
    src_band = src_ds.GetRasterBand(1)
    chunk_data = src_band.ReadAsArray(x, y, x_end - x, y_end - y)
    src_ds = None
    
    # Apply threshold
    mask = (chunk_data >= min_threshold) & (chunk_data <= max_threshold)
    chunk_data = np.where(mask, 1, 0)  # Reclassify
    
    # Resample to 10m resolution
    chunk_resampled = chunk_data.reshape(
        (chunk_data.shape[0] // 10, 10, chunk_data.shape[1] // 10, 10)
    ).sum(axis=(1, 3))  # Sum within 10x10 blocks
    
    return x, y, chunk_resampled


def process_large_raster(input_raster, output_raster, min_threshold, max_threshold, chunk_size=10000, compression="DEFLATE"):
    """Main function to process large raster in parallel."""
    # Open the input raster
    src_ds = gdal.Open(input_raster, gdalconst.GA_ReadOnly)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open input raster: {input_raster}")
    
    src_band = src_ds.GetRasterBand(1)
    nodata = src_band.GetNoDataValue()
    if nodata is None:
        nodata = 0
    
    # Get raster dimensions and geotransform
    xsize, ysize = src_ds.RasterXSize, src_ds.RasterYSize
    geotransform = src_ds.GetGeoTransform()
    projection = src_ds.GetProjection()
    
    # Prepare output raster
    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(
        output_raster,
        xsize // 10,  # Resampled to 10m resolution
        ysize // 10,
        1,
        gdalconst.GDT_Int32,
        options=["COMPRESS={}".format(compression)]
    )
    dst_ds.SetGeoTransform((
        geotransform[0], 10, 0, geotransform[3], 0, -10
    ))  # Update geotransform for 10m resolution
    dst_ds.SetProjection(projection)
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.SetNoDataValue(0)
    
    # Determine chunk size in pixels
    chunk_x_pixels = int(chunk_size // abs(geotransform[1]))  # Convert chunk size to pixels (meters -> pixels)
    chunk_y_pixels = int(chunk_size // abs(geotransform[5]))
    
    # Create tasks for parallel processing
    tasks = []
    for y in range(0, ysize, chunk_y_pixels):
        for x in range(0, xsize, chunk_x_pixels):
            x_end = min(x + chunk_x_pixels, xsize)
            y_end = min(y + chunk_y_pixels, ysize)
            tasks.append((input_raster, x, y, x_end, y_end, min_threshold, max_threshold, nodata))
    
    # Use multiprocessing to process chunks in parallel
    print(f"Using {cpu_count()} processors...")
    with Pool(cpu_count()) as pool:
        results = pool.map(process_chunk_parallel, tasks)
    
    # Combine results into the output array
    output_data = np.zeros((ysize // 10, xsize // 10), dtype=np.int32)
    for x, y, chunk_resampled in results:
        res_y_start = y // 10
        res_x_start = x // 10
        output_data[res_y_start:res_y_start + chunk_resampled.shape[0], 
                    res_x_start:res_x_start + chunk_resampled.shape[1]] += chunk_resampled
    
    # Write output raster
    dst_band.WriteArray(output_data)
    dst_band.FlushCache()
    dst_ds = None
    src_ds = None
    print(f"Processing complete: {output_raster}")


# Example usage
input_raster = "output.vrt"
output_raster = "lpis_10m.tif"

# Define threshold range
min_threshold = 1
max_threshold = 6666

process_large_raster(input_raster, output_raster, min_threshold, max_threshold)


Processing raster: ./chunked_rasters/chunk_270.tif
Processing raster: ./chunked_rasters/chunk_426.tif
Processing raster: ./chunked_rasters/chunk_203.tif
Processing raster: ./chunked_rasters/chunk_405.tif
Processing raster: ./chunked_rasters/chunk_202.tif
Processing raster: ./chunked_rasters/chunk_21.tif
Processing raster: ./chunked_rasters/chunk_178.tif
Processing raster: ./chunked_rasters/chunk_159.tif
Processing raster: ./chunked_rasters/chunk_347.tif
Processing raster: ./chunked_rasters/chunk_501.tif


ValueError: operands could not be broadcast together with shapes (1000,1000) (1000,871) (1000,1000) 

In [4]:
import os
import glob
import numpy as np
from osgeo import gdal, gdalconst
from multiprocessing import Pool, cpu_count

def process_chunk_parallel(args):
    """Worker function to process a single chunk in parallel."""
    src_raster, x, y, x_end, y_end, min_threshold, max_threshold, nodata = args
    src_ds = gdal.Open(src_raster, gdalconst.GA_ReadOnly)
    src_band = src_ds.GetRasterBand(1)
    chunk_data = src_band.ReadAsArray(x, y, x_end - x, y_end - y)
    src_ds = None
    
    # Apply threshold
    mask = (chunk_data >= min_threshold) & (chunk_data <= max_threshold)
    chunk_data = np.where(mask, 1, 0)  # Reclassify
    
    # Resample to 10m resolution
    chunk_resampled = chunk_data.reshape(
        (chunk_data.shape[0] // 10, 10, chunk_data.shape[1] // 10, 10)
    ).sum(axis=(1, 3))  # Sum within 10x10 blocks
    
    return x, y, chunk_resampled

def process_single_raster(input_raster, min_threshold, max_threshold, chunk_size, target_geotransform, target_shape):
    """Processes a single raster and returns the aligned resampled output."""
    src_ds = gdal.Open(input_raster, gdalconst.GA_ReadOnly)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open input raster: {input_raster}")
    
    src_band = src_ds.GetRasterBand(1)
    nodata = src_band.GetNoDataValue()
    if nodata is None:
        nodata = 0
    
    xsize, ysize = src_ds.RasterXSize, src_ds.RasterYSize
    geotransform = src_ds.GetGeoTransform()
    
    chunk_x_pixels = int(chunk_size // abs(geotransform[1]))
    chunk_y_pixels = int(chunk_size // abs(geotransform[5]))
    
    tasks = []
    for y in range(0, ysize, chunk_y_pixels):
        for x in range(0, xsize, chunk_x_pixels):
            x_end = min(x + chunk_x_pixels, xsize)
            y_end = min(y + chunk_y_pixels, ysize)
            tasks.append((input_raster, x, y, x_end, y_end, min_threshold, max_threshold, nodata))
    
    print(f"Processing raster: {input_raster}")
    with Pool(cpu_count()) as pool:
        results = pool.map(process_chunk_parallel, tasks)
    
    output_data = np.zeros((ysize // 10, xsize // 10), dtype=np.int32)
    for x, y, chunk_resampled in results:
        res_y_start = y // 10
        res_x_start = x // 10
        output_data[res_y_start:res_y_start + chunk_resampled.shape[0], 
                    res_x_start:res_x_start + chunk_resampled.shape[1]] += chunk_resampled
    
    # Align to target extent and shape
    aligned_data = align_to_target(output_data, geotransform, target_geotransform, target_shape, nodata=0)
    return aligned_data


def align_to_target(data, source_geotransform, target_geotransform, target_shape, nodata=0):
    """Aligns raster data to a target geotransform and shape by padding or cropping."""
    src_x_offset = int((source_geotransform[0] - target_geotransform[0]) / target_geotransform[1])
    src_y_offset = int((source_geotransform[3] - target_geotransform[3]) / target_geotransform[5])
    
    aligned_data = np.full(target_shape, nodata, dtype=np.int32)
    ysize, xsize = data.shape
    aligned_data[src_y_offset:src_y_offset + ysize, src_x_offset:src_x_offset + xsize] = data
    return aligned_data


def process_folder_rasters(input_folder, output_raster, min_threshold, max_threshold, chunk_size=10000, compression="DEFLATE"):
    """Processes all rasters in a folder and merges results."""
    raster_files = glob.glob(os.path.join(input_folder, "*.tif"))
    if not raster_files:
        raise FileNotFoundError(f"No rasters found in folder: {input_folder}")
    
    # Determine the target extent and shape from the first raster
    first_raster = gdal.Open(raster_files[0], gdalconst.GA_ReadOnly)
    target_geotransform = first_raster.GetGeoTransform()
    target_xsize = first_raster.RasterXSize // 10
    target_ysize = first_raster.RasterYSize // 10
    target_shape = (target_ysize, target_xsize)
    first_raster = None
    
    combined_data = np.zeros(target_shape, dtype=np.int32)
    
    for raster_file in raster_files:
        output_data = process_single_raster(raster_file, min_threshold, max_threshold, chunk_size, target_geotransform, target_shape)
        
        # Ensure the raster dimensions match the combined data before adding
        if output_data.shape != combined_data.shape:
            raise ValueError(f"Shape mismatch: {output_data.shape} vs {combined_data.shape}")
        
        combined_data += output_data  # Merge the data
    
    # Write the combined raster to file
    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(
        output_raster,
        target_shape[1],
        target_shape[0],
        1,
        gdalconst.GDT_Int32,
        options=["COMPRESS={}".format(compression)]
    )
    dst_ds.SetGeoTransform((
        target_geotransform[0], 10, 0, target_geotransform[3], 0, -10
    ))
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.SetNoDataValue(0)
    dst_band.WriteArray(combined_data)
    dst_band.FlushCache()
    dst_ds = None
    print(f"Processing complete: {output_raster}")


# Example usage
input_folder = "./chunked_rasters"  # Folder containing chunked rasters
output_raster = "LPIS_chunked_10m.tif"  # Output raster file

# Define threshold range
min_threshold = 1
max_threshold = 300

process_folder_rasters(input_folder, output_raster, min_threshold, max_threshold)


Processing raster: ./chunked_rasters/chunk_270.tif
Processing raster: ./chunked_rasters/chunk_426.tif


ValueError: could not broadcast input array from shape (1000,1000) into shape (0,0)